#  RUN THESE FIELDS TO PREPARE NECESSARY LIBRARIES AND DATA

In [94]:
#downloading all required libraries
import pandas as pd
import nltk
import os
nltk.download('punkt')
import difflib
from nltk.stem import PorterStemmer 
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/nazaryaremko/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [89]:
#this creates a map of names to convert them from Russian to Ukrainian and from Ukrainian to Russian
name_map = '/Users/nazaryaremko/Desktop/Names mapping.xlsx'
name_df = pd.read_excel(name_map)
name_dic_ru_to_ua = {}
name_dic_ua_to_ru = {}
for i in range(len(name_df)):
    name_dic_ru_to_ua[name_df.iloc[i][2]] = name_df.iloc[i][1]
    name_dic_ua_to_ru[name_df.iloc[i][1]] = name_df.iloc[i][2]

In [56]:
#HOW IT LOOKS LIKE
print(name_dic_ru_to_ua)

{'Александр': nan, 'Анна': 'Анна', 'Сергей': nan, 'Дмитрий': 'Дмитро', 'Ольга': 'Ольга', 'Андрей': 'Андрій', 'Юлия': 'Юлія', 'Анастасия': 'Анастасія', 'Максим': 'Максим', 'Татьяна': 'Тетяна', 'Елена': 'Олена', 'Евгений': 'Євген', 'Ирина': nan, 'Оксана': 'Оксана', 'Владислав': nan, 'Роман': 'Роман', 'Алексей': 'Олексій', 'Олег': 'Олег', 'Марина': 'Марина', 'Екатерина': 'Катерина', 'Виктория': nan, 'Денис': 'Денис', 'Игорь': 'Ігор', 'Владимир': nan, 'Наталья': nan, 'Виталий': 'Віталій', 'Богдан': 'Богдан', 'Антон': nan, 'Яна': 'Яна', 'Юрий': nan, 'Алина': 'Аліна', 'Артём': 'Артем', 'Светлана': 'Світлана', 'Мария': 'Марія', 'Вадим': 'Вадим', 'Людмила': 'Людмила', 'Дарья': 'Дарія', 'Николай': nan, 'Ярослав': 'Ярослав', 'Михаил': 'Михайло', 'Иван': 'Іван', 'Александра': 'Олександра', 'Руслан': 'Руслан', 'Павел': 'Павло', 'Валерия': 'Валерія', 'Инна': 'Інна', 'Константин': nan, 'Вячеслав': "В'ячеслав", 'Тарас': 'Тарас', 'Галина': 'Галина', 'Никита': nan, 'Алёна': 'Альона', 'Валентина': 'Вале

# BELOW ARE ALL THE COMPLEMENTARY FUNCTIONS USED (RUN THEM EVVERY TIME YOU OPEN THIS NOTEBOOK)

In [57]:
# [x,0] = the last name row
# [x,1] = the first name row - if first/last names are not the same then not a match, if the same, could be a match
                                #this will serve as a starting point 
# [x,2] = the job title row - if not the same, the person could change the profession, if the same medium likelihood
                                #that it is a same person
# [x,4] = the birthday row -> if not the same (more than a year difference) then not the same person
                                #if the same, almost certainly it is a duplicate
# [x,5] = the country row - if not the same, could still be the same person (relocated), if the same, doesn't tell much
# [x,6] = the location row -> if not the same, could still be the same person (relocated), if the same, somewhat likely
# [x,7] = the full name row  -> if not the same then definitely not a match and if the same, highly likely
#print(df.iloc[2,1]==df.iloc[3,1])


def find_duplicates(row_number):
    duplicates_arr = ["" for x in range(len(df))]
    sims = [[] for x in range(len(df))]
    for r in range(len(df)):
        if row_number == r:
            sims[r] = []
        else:
            sim = []
            #CHECKING LAST NAME SIMILARITY
            if df.iloc[row_number,0] == df.iloc[r,0]:
                sim.append('same')
            #here if the last name are at least 75% similar, then we append 'almost the same'
            #WARNING: can accidentally miss small last names because even one letter difference 
            #will cause a significant decrease in similarity AND can accidentally return that two long 
            #last names are similar because many a big number of different letters will not seem that
            #significant compared to the length of the entire last name
            elif last_name_similarity(df.iloc[row_number,0], df.iloc[r,0]) > 0.75:
                sim.append('almost the same')
            else: 
                sim.append('not same')
                duplicates_arr[r] = 'not a duplicate'
                continue
            #CHECKING FIRST NAME SIMILARITY
            if df.iloc[row_number,1] == df.iloc[r,1]:
                sim.append('same')
            #here we are checking if the russian version of the name is the same to current name
            #or if ukrainian version of the name is the same to current name (since we can't know)
            #if it's russian or ukrainian, so we check both
            elif df.iloc[row_number, 1] in name_dic_ru_to_ua and df.iloc[r, 1] == name_dic_ru_to_ua[df.iloc[row_number, 1]]:
                sim.append('same')
            elif df.iloc[row_number, 1] in name_dic_ua_to_ru and df.iloc[r, 1] == name_dic_ua_to_ru[df.iloc[row_number, 1]]: 
                sim.append('same')
            else: 
                sim.append('not same')
                duplicates_arr[r] = 'not a duplicate'
                continue
            #CHECKING FOR PARENTAL NAME
            if type(df.iloc[row_number,7]) != str or type(df.iloc[r,7]) != str or n_words(df.iloc[row_number,7])<3 or n_words(df.iloc[r,7])<3:
                sim.append('no paternal name')
            elif df.iloc[row_number,7].split(' ')[-1] == df.iloc[r,7].split(' ')[-1]:
                sim.append('same')
            else: 
                sim.append('not same')
                duplicates_arr[r] = 'not a duplicate'
                continue
            #CHECKING FOR BIRTHDATE
            if df.iloc[row_number,4] == df.iloc[r,4]:
                sim.append('exactly the same')
            #here we make sure that both dates are available
            elif pd.isnull(df.iloc[row_number,4]) or pd.isnull(df.iloc[r,4]):
                sim.append('one or two dates are missing')
            #here we check if the difference is at most 1 year
            elif birthday_diff(df.iloc[row_number,4], df.iloc[r,4]) < 2:
                sim.append('almost the same')
            else: 
                sim.append('not same')
                duplicates_arr[r] = 'not a duplicate'
                continue
            #CHECKING FOR TITLE SIMILARITY
            if df.iloc[row_number,2] == df.iloc[r,2]:
                sim.append('exactly the same')
            elif find_root(df.iloc[row_number,2], df.iloc[r,2]) > 0:
                sim.append('almost the same with ' + str(find_root(df.iloc[row_number,2], df.iloc[r,2]))+ ' common words')
            else: 
                sim.append('No common words')
            sims[r] = sim
    dup_fin = similarity_score(sims)
    return dup_fin

In [58]:
#here if you want to check a particular row, enter the row number here
#Python starts row indexing form 0, so to make sure you are looking at the right person, input row number in
#both print statements (it's normally row number in the original excel file - 2)
print(df.iloc[34,0], df.iloc[34,1])
print(find_duplicates(34))

Загоровець Ярослав
Somewhat  likely to be a duplicate to row 33 by first and last names; and by job title 3 common words


In [59]:
#this function produces the actualy entries to the column we will produce
#the way it works: it inserts "row x likely to be a duplicate to row y" if the name and last names 
#in rows x and y are the same. Further, if brithday, title are the same or similary it appends Highly or
#Somewhat likely to make the result more precise. If in the end, only last and first names were the same, then
#we do not consider this a duplicate (because at least birthday or title need to be similar)
def similarity_score(sims):
    for i in range(len(sims)):
        return_string = ''
        if sims[i] != []:
            if sims[i][0] == 'same' and sims[i][1] == 'same':
                return_string += ' likely to be a duplicate to row ' + str(i) + ' by first and last names;'
            elif sims[i][0] == 'almost the same' and sims[i][1] == 'same':
                return_string += ' could be a duplicate to row ' + str(i) + ' by first and last names (not exact match);'
            if sims[i][2] == 'same':
                return_string += '(+ match by paternal name)'
            if sims[i][3] == 'exactly the same':
                return_string = 'Highly ' + return_string + ' and by birthdate (exactly the same)'
            if sims[i][3] == 'almost the same':
                return_string = 'Somewhat ' + return_string + ' and by birthdate (almost the same)'
            if sims[i][4] == 'exactly the same':
                return_string = 'Highly ' + return_string + ' and by job title (exactly the same)'
            if sims[i][4][:6] == 'almost':
                return_string = 'Somewhat ' + return_string + ' and by job title ' + sims[i][4][21:]
        
        #here if it's only first/last names, I am not adding anything
        if return_string[:3] == ' li':
            sims[i] = ''
        else:
            sims[i] = return_string
    return " ".join(sims).strip()
        
# same name, last name, paternal name, exact same birthday, country and city - definitely a duplicate
# same name, last name, paternal name, almost the same birthday, country and city - highly likely a duplicate
# same name, similar last name, same birthday - highly likely to be a duplicate
# same name, almost the same birthday - potentially a duplicate

In [60]:
#this returns a similarity score between last names 
#later in the program, if similarity is abvoe 0.75, i consider them almost the same
def last_name_similarity(ln1, ln2):
    similarity = difflib.SequenceMatcher(None, ln1,ln2).ratio()
    return similarity

In [61]:
#this finds the numbers of words with the same root in the title
def find_root(title1, title2):
    ps = PorterStemmer()
    
    if type(title1) == float or type(title2) == float:
        return 0
    
    else:
        title1_stem = []
        for i in word_tokenize(title1):
            title1_stem.append(ps.stem(i.lower()))
        title2_stem = []
        for i in word_tokenize(title2):
            title2_stem.append(ps.stem(i.lower()))
        common_words = set(title1_stem).intersection(set(title2_stem))
        return len(common_words)

In [83]:
#this function finds the difference between years of birth
#IMPORTANT THE DATE MUST BE IN THE FOLLOWING FORMAT = 'January 1, 1986' 
def birthday_diff(date1, date2):
    year1 = int(str(date1).split(' ')[-1])
    year2 = int(str(date2).split(' ')[-1])
    diff = abs(year1 - year2)
    return diff

In [84]:
#EXAMPLE
birthday_diff('January 1, 1986', 'January 1, 1976')

10

In [64]:
#this calculates the number of words in a sentence
def n_words(word):
    a_string = word
    word_list = a_string.split()

    return len(word_list)

#  RUN THESE FIELDS TO PRODUCE THE RESULTS

###  STEP ONE - DOWNLOAD EXCEL FILE INTO A DATAFRAME

The excel file must have the columns in the following order (filds that are required are in bold):  
<b>LAST NAME</b> | <b>FIRST NAME</b> | TITLE (in English) | ID | <b>BIRTHDAY</b> | COUNTRY | LOCATION | CYRILLIC FULL NAME  



IMPORTANT - the date must be in a specific format AS IN THE EXCEL FILE EXAMPLE I PROVIDED

In [90]:
#here we put in the path to the excel file we want
#it also prints the head of the file to make sure that the file is correct 
file_errors_location = '/Users/nazaryaremko/Desktop/EXCEL FILE TEMPLATE.xlsx'
df = pd.read_excel(file_errors_location)
print(df.head(5))

  Last Name First Name   Translate CustomTitle  Candidate ID         Birthday  \
0     Adeel        Ali              Accountant   55097954002              NaN   
1     Adeel        Ali              Accountant   54808308002  January 1, 1991   
2      Adel    Mohamed              Accountant   70279083002  January 1, 1998   
3    Aliyev      Kanan  Accountant, translator   70763946002  January 1, 1995   
4  Adjabeng     Philip           Administrator   63031219002  January 1, 1994   

   Country Custom location Cyrillic full name  
0  Ukraine            Kyiv                NaN  
1  Ukraine            Kyiv                NaN  
2  Ukraine           Odesa                NaN  
3  Ukraine            Kyiv                NaN  
4  Ukraine            Kyiv                NaN  


### STEP TWO - CREATE A NEW COLUMN WITH THE RESULTS (THIS MIGHT RUN FOR A WHILE)

In [85]:
#here we create a list of all results for each row
new_long_column = [[] for _ in range(len(df))]

for i in range(len(df)):
    new_long_column[i].append(find_duplicates(i))

print(new_long_column)

[['Highly  likely to be a duplicate to row 1 by first and last names; and by job title (exactly the same)'], ['Highly  likely to be a duplicate to row 0 by first and last names; and by job title (exactly the same)'], [''], ['Highly  likely to be a duplicate to row 56 by first and last names; and by birthdate (exactly the same)'], ['Somewhat  likely to be a duplicate to row 26 by first and last names; and by birthdate (almost the same)'], [''], [''], ['Somewhat  likely to be a duplicate to row 66 by first and last names; and by job title 1 common words'], ['Somewhat  likely to be a duplicate to row 64 by first and last names; and by birthdate (almost the same)       Somewhat Somewhat  likely to be a duplicate to row 71 by first and last names; and by birthdate (almost the same) and by job title 1 common words'], [''], ['Somewhat Somewhat  likely to be a duplicate to row 69 by first and last names; and by birthdate (almost the same) and by job title 1 common words'], [''], [''], ['Somewh

### STEP THREE - APPEND A NEW COLUMN TO YOUR DATAFRAME

In [101]:
#Here we append a new column to our dataframe (excel table that we saved in Python)
df['Duplicates'] = new_long_column

### STEP FOUR - DOWNLOAD THE OUTPUT FILE TO YOUR COMPUTER (THE NAME WILL BE OUTPUT.XLSX)

In [105]:
# create excel writer object
#you can change the name in the line below if you want
df.to_excel(r'/Users/nazaryaremko/Desktop\MY RESULT.xlsx', index = False)
print('DataFrame is written successfully to Excel File.')

DataFrame is written successfully to Excel File.
